In [3]:
from datetime import datetime, timedelta, timezone
import json
import os
import re
import boto3
from time import sleep
from threading import Thread

import pandas as pd

from sagemaker import get_execution_role, session, Session, image_uris
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.processing import ProcessingJob
from sagemaker.serializers import CSVSerializer

from sagemaker.model import Model
from sagemaker.model_monitor import DataCaptureConfig

session = Session()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
role = get_execution_role()
print("RoleArn:", role)

region = session.boto_region_name
print("Region:", region)

RoleArn: arn:aws:iam::364723530364:role/LabRole
Region: us-east-1


In [5]:
bucket = session.default_bucket()
print(bucket)

sagemaker-us-east-1-364723530364


In [6]:
import pandas as pd

In [7]:
!ls "Car Data"

audi.csv  cclass.csv  ford.csv	  merc.csv   toyota.csv    vw.csv
bmw.csv   focus.csv   hyundi.csv  skoda.csv  vauxhall.csv


In [8]:
audi = pd.read_csv('Car Data/audi.csv')
cclass = pd.read_csv('Car Data/cclass.csv')
ford = pd.read_csv('Car Data/ford.csv')
merc = pd.read_csv('Car Data/merc.csv')
toyota = pd.read_csv('Car Data/toyota.csv')
vw = pd.read_csv('Car Data/vw.csv')
bmw = pd.read_csv('Car Data/bmw.csv')
focus = pd.read_csv('Car Data/focus.csv')
hyundi = pd.read_csv('Car Data/hyundi.csv')
skoda = pd.read_csv('Car Data/skoda.csv')
vauxhall = pd.read_csv('Car Data/vauxhall.csv')

In [9]:
datasets = [audi, cclass, ford, merc, toyota, vw, bmw, focus, hyundi, skoda, vauxhall]

In [10]:
makers = ["audi", "cclass", "ford", "merc", "toyota", "vw", "bmw", "focus", "hyundi", "skoda", "vauxhall"]

In [11]:
hyundi.rename(columns = {'tax(£)':'tax'}, inplace = True)


### Checking for Nulls in any of the datasets

In [12]:
for i,data in enumerate(datasets):
    print(data.isnull().sum().sum())
    data["maker"] = makers[i] 
#There are no null values

0
0
0
0
0
0
0
0
0
0
0


In [13]:
df_merged = pd.concat(datasets, axis=0)

In [14]:
df_merged

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,maker
0,A1,2017,12500,Manual,15735,Petrol,150.0,55.4,1.4,audi
1,A6,2016,16500,Automatic,36203,Diesel,20.0,64.2,2.0,audi
2,A1,2016,11000,Manual,29946,Petrol,30.0,55.4,1.4,audi
3,A4,2017,16800,Automatic,25952,Diesel,145.0,67.3,2.0,audi
4,A3,2019,17300,Manual,1998,Petrol,145.0,49.6,1.0,audi
...,...,...,...,...,...,...,...,...,...,...
13627,Tigra,2007,2499,Manual,48640,Petrol,165.0,46.3,1.4,vauxhall
13628,Vectra,2007,1795,Manual,75000,Petrol,240.0,39.2,1.8,vauxhall
13629,Vectra,2005,1495,Manual,74000,Petrol,235.0,38.7,1.8,vauxhall
13630,Vectra,2008,1695,Automatic,131000,Diesel,200.0,39.8,1.9,vauxhall


In [15]:
df_merged.isnull().sum()

model              0
year               0
price              0
transmission       0
mileage            0
fuelType           0
tax             9353
mpg             9353
engineSize         0
maker              0
dtype: int64

### Handeling Missing Values

In [16]:
df_merged['tax'].fillna(int(df_merged['tax'].mean()), inplace=True) #Filling them with mean
df_merged['mpg'].fillna(int(df_merged['mpg'].mean()), inplace=True) #Filling them with mean

/tmp/ipykernel_18/1043064067.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merged['tax'].fillna(int(df_merged['tax'].mean()), inplace=True) #Filling them with mean
/tmp/ipykernel_18/1043064067.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[

In [17]:
df_merged.isnull().sum() #Verify there is no more missing values

model           0
year            0
price           0
transmission    0
mileage         0
fuelType        0
tax             0
mpg             0
engineSize      0
maker           0
dtype: int64

### One Hot Encoding

In [18]:
df_merged

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,maker
0,A1,2017,12500,Manual,15735,Petrol,150.0,55.4,1.4,audi
1,A6,2016,16500,Automatic,36203,Diesel,20.0,64.2,2.0,audi
2,A1,2016,11000,Manual,29946,Petrol,30.0,55.4,1.4,audi
3,A4,2017,16800,Automatic,25952,Diesel,145.0,67.3,2.0,audi
4,A3,2019,17300,Manual,1998,Petrol,145.0,49.6,1.0,audi
...,...,...,...,...,...,...,...,...,...,...
13627,Tigra,2007,2499,Manual,48640,Petrol,165.0,46.3,1.4,vauxhall
13628,Vectra,2007,1795,Manual,75000,Petrol,240.0,39.2,1.8,vauxhall
13629,Vectra,2005,1495,Manual,74000,Petrol,235.0,38.7,1.8,vauxhall
13630,Vectra,2008,1695,Automatic,131000,Diesel,200.0,39.8,1.9,vauxhall


In [19]:
df_merged["model"].nunique() # This is to many categories to one hot encode 


195

In [20]:
High_Freq_models = df_merged["model"].value_counts()[:30]

In [21]:
High_Freq_models = High_Freq_models.index

In [22]:
df_merged.loc[~df_merged["model"].isin(High_Freq_models), "model"] = "Other" # Replace all not in the most Likely 30 with Other

In [32]:
df_encoded = pd.get_dummies(df_merged, columns=['model'])

In [33]:
df_encoded

,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,maker,model_ 1 Series,...,model_ Insignia,model_ Kuga,model_ Mokka X,model_ Octavia,model_ Polo,model_ Q3,model_ Tiguan,model_ Tucson,model_ Yaris,model_Other
0,2017,12500,Manual,15735,Petrol,150.0,55.4,1.4,audi,False,...,False,False,False,False,False,False,False,False,False,False
1,2016,16500,Automatic,36203,Diesel,20.0,64.2,2.0,audi,False,...,False,False,False,False,False,False,False,False,False,True
2,2016,11000,Manual,29946,Petrol,30.0,55.4,1.4,audi,False,...,False,False,False,False,False,False,False,False,False,False
3,2017,16800,Automatic,25952,Diesel,145.0,67.3,2.0,audi,False,...,False,False,False,False,False,False,False,False,False,False
4,2019,17300,Manual,1998,Petrol,145.0,49.6,1.0,audi,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13627,2007,2499,Manual,48640,Petrol,165.0,46.3,1.4,vauxhall,False,...,False,False,False,False,False,False,False,False,False,True
13628,2007,1795,Manual,75000,Petrol,240.0,39.2,1.8,vauxhall,False,...,False,False,False,False,False,False,False,False,False,True
13629,2005,1495,Manual,74000,Petrol,235.0,38.7,1.8,vauxhall,False,...,False,False,False,False,False,False,False,False,False,True
13630,2008,1695,Automatic,131000,Diesel,200.0,39.8,1.9,vauxhall,False,...,False,False,False,False,False,False,False,False,False,True


In [34]:
df_encoded = pd.get_dummies(df_encoded, columns=['maker']) #Replace maker with one hot encoded

In [35]:
df_encoded = pd.get_dummies(df_encoded, columns=['transmission']) #Replace transmission with one hot encoded

In [36]:
df_encoded = pd.get_dummies(df_encoded, columns=['fuelType']) #Replace fueltype with one hot encoded

In [37]:
first_column = df_encoded.pop('price') 
df_encoded.insert(0, 'price', first_column) 

In [38]:
df_encoded = df_encoded.drop(["tax"], axis=1) #Price and Tax are correlated
df_encoded

,price,year,mileage,mpg,engineSize,model_ 1 Series,model_ 2 Series,model_ 3 Series,model_ 4 Series,model_ 5 Series,...,maker_vw,transmission_Automatic,transmission_Manual,transmission_Other,transmission_Semi-Auto,fuelType_Diesel,fuelType_Electric,fuelType_Hybrid,fuelType_Other,fuelType_Petrol
0,12500,2017,15735,55.4,1.4,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
1,16500,2016,36203,64.2,2.0,False,False,False,False,False,...,False,True,False,False,False,True,False,False,False,False
2,11000,2016,29946,55.4,1.4,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
3,16800,2017,25952,67.3,2.0,False,False,False,False,False,...,False,True,False,False,False,True,False,False,False,False
4,17300,2019,1998,49.6,1.0,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13627,2499,2007,48640,46.3,1.4,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
13628,1795,2007,75000,39.2,1.8,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
13629,1495,2005,74000,38.7,1.8,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
13630,1695,2008,131000,39.8,1.9,False,False,False,False,False,...,False,True,False,False,False,True,False,False,False,False


In [63]:
df_encoded = df_encoded.replace({True: 1, False: 0})
df_encoded

/tmp/ipykernel_18/3639597318.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_encoded = df_encoded.replace({True: 1, False: 0})


,price,year,mileage,mpg,engineSize,model_ 1 Series,model_ 2 Series,model_ 3 Series,model_ 4 Series,model_ 5 Series,...,maker_vw,transmission_Automatic,transmission_Manual,transmission_Other,transmission_Semi-Auto,fuelType_Diesel,fuelType_Electric,fuelType_Hybrid,fuelType_Other,fuelType_Petrol
0,12500,2017,15735,55.4,1.4,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1,16500,2016,36203,64.2,2.0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
2,11000,2016,29946,55.4,1.4,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,16800,2017,25952,67.3,2.0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
4,17300,2019,1998,49.6,1.0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13627,2499,2007,48640,46.3,1.4,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
13628,1795,2007,75000,39.2,1.8,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
13629,1495,2005,74000,38.7,1.8,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
13630,1695,2008,131000,39.8,1.9,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0


### Train Test and batch splitting

In [64]:
import numpy as np
rand_split = np.random.rand(len(df_encoded))
train_list = rand_split < 0.8
val_list = (rand_split >= 0.8) & (rand_split < 0.9)
batch_list = rand_split >= 0.9

In [65]:
data_train = df_encoded[train_list]
data_val = df_encoded[val_list]
data_batch = df_encoded[batch_list].drop(["price"], axis=1)

In [66]:
train_file = "train_data.csv"
data_train.to_csv(train_file, index=False, header=False)

In [67]:
validation_file = "validation_data.csv"
data_val.to_csv(validation_file, index=False, header=False)

In [68]:
batch_file = "batch_data.csv"
data_batch.to_csv(batch_file, index=False, header=False)

### Uploading to S3

In [69]:
import os
import boto3
import sagemaker

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name

bucket = sess.default_bucket()
prefix = "xgboost-final"

In [70]:
training_data = sess.upload_data(train_file, key_prefix="{}/train".format(prefix))
val_data = sess.upload_data(validation_file, key_prefix="{}/validation".format(prefix))
batch_data = sess.upload_data(batch_file, key_prefix="{}/batch".format(prefix))

In [71]:
training_data

's3://sagemaker-us-east-1-364723530364/xgboost-final/train/train_data.csv'

### Example Xgb Model

In [72]:
job_name = "Training_Job"

In [73]:
sm_estimator.hyperparameters()

{'objective': 'reg:squarederror',
 'max_depth': 5,
 'subsample': 0.8,
 'verbosity': 0,
 'num_round': 1000}

In [77]:
from time import gmtime, strftime

job_name = "xgb-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
output_location = "s3://{}/{}/output/{}".format(bucket, prefix, job_name)
image = sagemaker.image_uris.retrieve(
    framework="xgboost", region=boto3.Session().region_name, version="1.7-1"
)

sm_estimator = sagemaker.estimator.Estimator(
    image,
    role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size=50,
    input_mode="File",
    output_path=output_location,
    sagemaker_session=sess,
)

sm_estimator.set_hyperparameters(
    objective="reg:squarederror",
    max_depth=5,
    subsample=0.8,
    verbosity=0,
    eta = .4,
    num_round=250,
)

train_data = sagemaker.inputs.TrainingInput(
    training_data,
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
)
validation_data = sagemaker.inputs.TrainingInput(
    val_data,
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
)
data_channels = {"train": train_data, "validation": validation_data}

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [78]:
sm_estimator.fit(inputs=data_channels, job_name=job_name, logs=True)

INFO:sagemaker:Creating training-job with name: xgb-2024-06-19-03-26-32


2024-06-19 03:26:33 Starting - Starting the training job...
2024-06-19 03:26:48 Starting - Preparing the instances for training...
2024-06-19 03:27:16 Downloading - Downloading input data...
2024-06-19 03:27:41 Downloading - Downloading the training image...
2024-06-19 03:28:22 Training - Training image download completed. Training in progress....
2024-06-19 03:29:02 Uploading - Uploading generated training model[2024-06-19 03:28:33.076 ip-10-2-72-85.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-06-19 03:28:33.097 ip-10-2-72-85.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-06-19:03:28:33:INFO] Imported framework sagemaker_xgboost_container.training
[2024-06-19:03:28:33:INFO] Failed to parse hyperparameter objective value reg:squarederror to Json.
Returning the value itself
[2024-06-19:03:28:33:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-19:03:28:33:INFO] Running XGBoost Sagemaker in algorithm mode
[20

### Batch Trasnform

In [79]:
batch_data

's3://sagemaker-us-east-1-364723530364/xgboost-final/batch/batch_data.csv'

In [80]:
sm_transformer = sm_estimator.transformer(1, "ml.m4.xlarge")
sm_transformer.transform(batch_data, content_type="text/csv", split_type="Line")
sm_transformer.wait()

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-06-19-03-35-18-350
INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2024-06-19-03-35-19-150


.........................................[2024-06-19:03:42:09:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-19:03:42:09:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-19:03:42:09:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
[2024-06-19:03:42:09:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-19:03:42:09:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-19:03:42:09:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|

In [83]:
s3 = boto3.client("s3")

In [96]:
sm_transformer.output_path

's3://sagemaker-us-east-1-364723530364/sagemaker-xgboost-2024-06-19-03-35-19-150'

In [99]:
transform_df = pd.read_csv(sm_transformer.output_path + "/batch_data.csv.out")  

In [100]:
transform_df

,13200.380859375
0,19245.064453
1,19966.634766
2,15140.334961
3,17870.853516
4,16351.642578
...,...
10867,14321.133789
10868,15451.561523
10869,15166.341797
10870,8775.341797


In [108]:
sagemaker = boto3.client("sagemaker")

model_name = job_name

info = sagemaker.describe_training_job(TrainingJobName=model_name)
model_data = info["ModelArtifacts"]["S3ModelArtifacts"]

primary_container = {"Image": image, "ModelDataUrl": model_data}

create_model_response = sagemaker.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
)
print(create_model_response["ModelArn"])

arn:aws:sagemaker:us-east-1:364723530364:model/xgb-2024-06-19-03-26-32


### 